<a href="https://colab.research.google.com/github/gatienc/multimodal_product_data_classification/blob/main/notebooks/gatien_fusion_model_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [327]:
!pip install unidecode

#Idea
idea of this fusion model: our 2 model gives a vector of 27 element as output,
we want to freeze both pretrained model, and just combine the output to classify better than on by one

il faudrait qu'il y ai une couche qui fasse : $a*out_{image}+b*out_{text}$
avec a et b des params entraînables

In [328]:
BATCH_SIZE=275#10.4/15GO
num_classes=27

train_percentage=0.80
test_percentage=0.10
valid_percentage=0.10


# Imports


In [329]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [330]:

import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
import torchvision

import imageio

from tqdm.auto import tqdm # for cool loading bar


# import for NLP
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.utils import to_categorical

from unidecode import unidecode
from nltk.corpus import stopwords



from sklearn.metrics import f1_score

import zipfile
import os
import copy

import matplotlib.pyplot as plt


In [331]:
# mount the drive where your dataset is availabledevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from google.colab import drive
drive.mount('/content/drive')
filepath='/content/drive/MyDrive/datasets/multimodal_product_classification/' # add your own path. Where to save the dataset

if not os.path.exists('datasets'):
  os.makedirs('datasets')
  with zipfile.ZipFile(filepath+'images.zip', 'r') as zip_ref:
      zip_ref.extractall('datasets')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [332]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Preprocessing

In [333]:
# Load data
X_train = pd.read_csv(filepath+'X_train.csv')
y_train = pd.read_csv(filepath+'Y_train.csv')
X_train=X_train.drop(columns="Unnamed: 0")
y_train=y_train.drop(columns="Unnamed: 0")


In [334]:
# Cleaning and Preprocessing Text


CLEANR = re.compile('<.*?>') # delete html tag

def clean_html(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext
def clean_special(text):
    text = re.sub(r'[^a-z% ]', '', text)
    return(text)

def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    text=re.sub('\d', '%', text)#replace all digits by %
    text=re.sub("'", ' ', text)#replace all ' by a space

    text = unidecode(text)
    text=clean_html(text)
    text=clean_special(text)
    #clean stopwords
    text_list=text.split()
    text_list=[word for word in text_list if word not in final_stopwords_list]
    text=" ".join(text_list)
    return(text)



final_stopwords_list = stopwords.words('english') + stopwords.words('french')+stopwords.words('german')# Fr,En,De Stopwords


# Apply cleaning function to the 'designation' column
X_train['designation'] = X_train['designation'].fillna('').apply(clean_text)
X_train['description'] = X_train['description'].fillna('').apply(clean_text)


In [335]:
pd.set_option('display.max_colwidth', 150)
X_train.designation.head(50)


0                                                                            olivia personalisiertes notizbuch %%% seiten punktraster ca din a% rosendesign
1     journal arts ndeg %%% %%%%%%%% art marche salon art asiatique paris jacques barrere francois perrier reforme ventes encheres publiques sna fete ce...
2                                                                                grand stylet ergonomique bleu gamepad nintendo wii u speedlink pilot style
3                                                                                                   peluche donald europe disneyland %%%% marionnette doigt
4                                                                                                                                             guerre tuques
5                                                                                           afrique contemporaine ndeg %%% hiver %%%% dossier japon afrique
6                                                               

In [336]:
X_train.description.head(50)


0                                                                                                                                                          
1                                                                                                                                                          
2     pilot style touch pen marque speedlink % stylet ergonomique gamepad nintendo wii u confort optimal precision maximale gamepad wii u grand stylet h...
3                                                                                                                                                          
4           luc ideacutees grandeur veut organiser jeu guerre boules neige arranger ecirctre vainqueur incontesteacute sophie mecircle chambarde tous plans
5                                                                                                                                                          
6                                                               

In [337]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['designation'])
designation_sequences = tokenizer.texts_to_sequences(X_train['designation'])
description_sequences = tokenizer.texts_to_sequences(X_train['description'])


In [338]:
designation_maxlen=max(len(x) for x in designation_sequences)#maxlen for designation
designation_maxlen

40

In [ ]:
max(len(x) for x in tokenizer.texts_to_sequences(X_train['description']))#max description size is 1090


In [ ]:
import plotly.express as px
desc_len=[len(x) for x in tokenizer.texts_to_sequences(X_train['description'])]
fig = px.histogram(pd.DataFrame(desc_len,columns=["x"]), x="x")
fig.show()
#plt.hist(desc_len,)

In [ ]:
# Padding to max length of text
padded_designation=pad_sequences(designation_sequences,maxlen=designation_maxlen)
padded_description=pad_sequences(description_sequences,maxlen=200)

In [ ]:
for i, row in X_train.iterrows():
    X_train.at[i, 'designation'] = padded_designation[i]
    X_train.at[i, "description"] = padded_description[i]

In [ ]:
X_train

In [ ]:
# Assuming the number of unique words in the tokenizer plus 1 is vocab_size
vocab_size = len(tokenizer.word_index) + 1
print(f'{vocab_size=}')# nearly 64517 of vocab size, it seems too much :/

In [ ]:
# Split data into training and validation set (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2,shuffle=False)
y_train=y_train["prdtypecode"].tolist()
y_val=y_val["prdtypecode"].tolist()

# Convert labels to categorical
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# Convert to one-hot encoding
y_train_categorical = to_categorical(y_train_encoded)
y_val_categorical = to_categorical(y_val_encoded)

In [ ]:
y_train=pd.DataFrame(y_train,columns=["labels"])

In [ ]:
train_df=pd.concat([X_train,y_train],axis=1)

In [ ]:
train_df


# Data loading

In [ ]:
def image_to_tensor(image):
    img=torch.from_numpy(image)
    out=img.permute(2,0,1)
    return(out)

In [ ]:
class ImageTextDataLoader():
    """Title, Description and Image dataset."""

    def __init__(self, dataframe, image_dir, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            image_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        if type(idx)==str:
            idx=int(idx)

        selected_df=self.df.iloc[idx]

        for index, row in selected_df.iterrows():
            print(row)
            image_name="image_"+str(row["imageid"])+"_product_"+str(row["productid"])+".jpg"
            filepath=os.path.join(self.image_dir,image_name)
            print(filepath)
            image_arr = imageio.v3.imread(filepath)
            t_image=image_to_tensor(image_arr)
            if 't_images' in locals():
                torch.cat([t_images,t_image])
            else:
                t_images=t_image


        if self.transform:
           t_images = self.transform(t_images)

        t_designation=torch.tensor(selected_df['designation'].to_list(),device=device)
        t_description=torch.tensor(selected_df['description'].to_list(),device=device)
        t_labels=torch.tensor(selected_df['labels'],device=device)

        return [t_designation,t_description,t_images,t_labels]


In [ ]:
transform = torchvision.transforms.Compose([
            torchvision.transforms.Resize(256),
            torchvision.transforms.CenterCrop(224),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
       ])

In [ ]:
dataset=ImageTextDataLoader(train_df,"/contents/datasets/images/image_train",transform)


In [ ]:
dataset[[0,1]]

In [ ]:
train_size = int(train_percentage * len(dataset))
valid_size = int(test_percentage * len(dataset))
test_size = len(dataset) - train_size - valid_size

train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)


dataloaders={
    'train':train_dataloader,
    'val':valid_dataloader,
    'test':test_dataloader
    }

model = torchvision.models.resnet18(weights="IMAGENET1K_V1")

# Replace the last fully-connected layer
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, num_classes) # num_classes is the number of classes in your dataset


model.load_state_dict(torch.load("/content/drive/MyDrive/Lessons/Models/multimodal_classification/Resnet16-best10.ckpt",map_location=device))


In [ ]:
data_batch = next(iter(train_dataloader))

In [ ]:
data_batch